# Seleção de Features usando o teste chi quadrado de independência

## 1. Carregando módulos e dados

In [2]:
# Manipulação dos dados
import numpy  as np
import scipy  as sp
import pandas as pd

# Visualização de dados
import matplotlib.pyplot as plt
import seaborn           as sns
%matplotlib inline

In [3]:
# Carregando os dados
data = pd.read_excel('../../Dados/Banco labels.xlsx')

In [4]:
# Visualização da tabela
data.head(5)

,TB_desfecho_final_Dri,Tbafranio,Obs_desfechoTB_Dri_Dani,Data_inclusão,Nome,Desfecho_tuberculose_lista_Mauro,Desfecho_TB_162_Revisão_ADRIANA,Observaça0_desfecho_TB,Resultado_BAAR_1,Resultado_BAAR_2,...,Perdeu_aula_hoje_CA,Deixou_de_ganhar_dinheiro_hoje_CA,Quanto_deixou_ganhar_CA,Caso_acompanhado_quem_custeou_tudo_CA,Valor_CA,Avaliador_custo_CA,Data_avaliação_custo_CA,Resultado_cultura_final < 3 & Realizou_RX = 1 (FILTER),ANO de inclusao,Novocriteriotbafranio
0,TB,SIM,NaN,2008-04-03,NaN,9,TB,TB PROBABILIDADE,Negativo,Negativo,...,IGN,IGN,Ignorado,IGN,ignorado,9,1999-09-09,Selected,2008,1
1,TB,SIM,NaN,2006-10-03,NaN,TB,TB,NaN,Negativo,Negativo,...,IGN,IGN,Ignorado,IGN,ignorado,9,1999-09-09,Selected,2006,1
2,TB,SIM,NaN,2007-06-21,NaN,TB,TB,TB CONFIRMADA,Negativo,Negativo,...,IGN,IGN,Ignorado,IGN,ignorado,9,1999-09-09,Selected,2007,1
3,TB,SIM,NaN,2009-09-30,NaN,TB,TB,NaN,Negativo,Negativo,...,IGN,IGN,Ignorado,IGN,ignorado,9,1999-09-09,Selected,2009,1
4,TB,SIM,NaN,2009-06-19,NaN,TB,NaN,NaN,Negativo,Negativo,...,Não,Não,"""Não se aplica""",não se aplica,não se aplica,Marcia dos Santos,2009-06-19,Not Selected,2009,1


In [11]:
# Alterando o nome da variável TB_desfecho_final_Dri para desfecho
data.rename(columns={'TB_desfecho_final_Dri':'desfecho'},inplace=True)

In [12]:
# Carregando as features salvas na seleção inicial de variáveis
import pickle
features_selecionadas = pickle.load(open('../../Dados/features_filtradas.pkl', 'rb'))

In [13]:
# Exibindo as variáveis salvas
print(features_selecionadas)

# Criando um dataframe a partir das features pré-selecionadas
dados = data[features_selecionadas].copy()

['desfecho', 'TTO_anterior_TB_triagem_enfermeiro', 'Cicatriz_BCG', 'Idade', 'Sexo', 'Raça', 'Estado_civil', 'Tem_companheiro', 'Peso_habitual', 'Peso_atual', 'Altura_cm', 'Tosse', 'Semanas_tosse', 'Expectoração', 'Semanas_expectoração', 'Hemoptóicos', 'Semanas_hemoptóicos', 'Hemoptise', 'Semanas_hemoptise', 'Quantidade_hemoptise', 'Sudorese_noturna', 'Semanas_sudorese', 'Febre', 'Semanas_febre', 'Dispnéia', 'Dispnéia_semanas', 'Perda_de_apetite', 'Perda_apetite_semanas', 'Perda_peso_10percent', 'Perda_peso_semanas', 'Dor_torácica', 'Dor_torácica_semanas', 'Internação_hospital_2anos', 'Prisão_2anos', 'Delegacia_2anos', 'Contato_TBP_2anos', 'Fuma', 'Número_cigarros_dia', 'CAGE']


### 1.1 Pré-processamento dos dados

In [18]:
# Alterando os labels dos desfechos
dados.replace(['TB', 'Não TB','IGN',8],['TB+','TB-','desconhecido','desconhecido'],inplace=True)

# Selecionando os desfechos que possuem diagnóstico conhecido: TB+ e TB-
dados = dados[(dados.desfecho == 'TB+') & (dados.desfecho == 'TB-')]

In [20]:
# Verificando os tipos de dados presentes
dados.dtypes

desfecho                               object
TTO_anterior_TB_triagem_enfermeiro     object
Cicatriz_BCG                           object
Idade                                  object
Sexo                                   object
Raça                                   object
Estado_civil                           object
Tem_companheiro                        object
Peso_habitual                          object
Peso_atual                            float64
Altura_cm                             float64
Tosse                                  object
Semanas_tosse                          object
Expectoração                           object
Semanas_expectoração                   object
Hemoptóicos                            object
Semanas_hemoptóicos                    object
Hemoptise                              object
Semanas_hemoptise                      object
Quantidade_hemoptise                   object
Sudorese_noturna                       object
Semanas_sudorese                  

## 2. Seleção de variáveis utilizando o teste do chi quadrado de independência

O teste do chi quadrado verifica se existe a relação entre duas variáveis categóricas. Dessa forma, será verificada a relação entre as variáveis presentes e o desfecho, ou seja, se o atributo possui alguma relação com o diagnóstico da doença ou não. 

O teste do Chi-quadrado é realizado para a verificação de independência entre duas variáveis categóricas.

- Hipótese nula $H_0$ : As variáveis são independentes.
- Hipótese alternativa $H_a$: As variáveis são dependentes

In [28]:
# Substituíndo valores ignorados para valores não computados pela tabela de contingência
dados_filtrados.replace(['ignorado'],np.nan,inplace=True)

In [29]:
# Criando um DataFrame para o armazenamento dos valores

dict_features = dict()

for feature in list(dados_filtrados.columns)[1:]:
    
    if dados_filtrados[feature].dtype != 'O':
        continue
    
    print('Tabela de Contingência entre o desfecho e {}\n'.format(feature))
    
    # Contingency table from pandas
    cross_tab = pd.crosstab(dados_filtrados[feature],dados_filtrados['desfecho'],margins=True)
    print(pd.crosstab(dados_filtrados[feature],dados_filtrados['desfecho'],margins=True))
    print()
    
    # Chi Squared Test 
    chi2, p, dof, expected =  chi2_contingency(cross_tab)
    print('Chi2: {}\tp-value: {}\tDOF: {}\nExpected_val: \n{}\n\n'.format(chi2,p,dof,expected))
    
    dict_features[feature] = [chi2,p,dof,(len(dados_filtrados[feature].unique()) - 1),]

Tabela de Contingência entre o desfecho e TTO_anterior_TB_triagem_enfermeiro

desfecho                            TB+   TB-   All
TTO_anterior_TB_triagem_enfermeiro                 
Não                                 668  1388  2056
Sim                                 202   453   655
All                                 870  1841  2711

Chi2: 0.6210017762208534	p-value: 0.9607015629884483	DOF: 4
Expected_val: 
[[  659.80081151  1396.19918849  2056.        ]
 [  210.19918849   444.80081151   655.        ]
 [  870.          1841.          2711.        ]]


Tabela de Contingência entre o desfecho e Cicatriz_BCG

desfecho      TB+   TB-   All
Cicatriz_BCG                 
Não           222   614   836
Sim           434   780  1214
All           656  1394  2050

Chi2: 19.234217052577776	p-value: 0.0007068855301971453	DOF: 4
Expected_val: 
[[  267.52   568.48   836.  ]
 [  388.48   825.52  1214.  ]
 [  656.    1394.    2050.  ]]


Tabela de Contingência entre o desfecho e Sexo

desfecho   TB

In [30]:
# Criando um DataFrame a partir d
chi2_dict = pd.DataFrame.from_dict(dict_features,orient='index')
chi2_dict.columns = ['chi2','pval','dof','dof_real']

# Dataframe criado
chi2_dict

,chi2,pval,dof,dof_real
CAGE,9.007891,6.090254e-02,4,2
Delegacia_2anos,20.548447,3.890882e-04,4,2
Sexo,53.179802,1.079427e-09,6,3
Expectoração,3.315559,5.064709e-01,4,2
Raça,10.612917,3.127663e-02,4,2
Cicatriz_BCG,19.234217,7.068855e-04,4,2
Febre,92.249234,4.381080e-19,4,2
Prisão_2anos,39.277414,6.105206e-08,4,2
Fuma,9.230613,1.610179e-01,6,3
Perda_peso_10percent,201.841399,1.509940e-42,4,2


In [40]:
variaveis_dependentes = chi2_dict[chi2_dict.pval < 0.05]
variaveis_independentes = chi2_dict[chi2_dict.pval > 0.05]
len(variaveis_dependentes)
#len(variaveis_independentes)

14

In [37]:
variaveis_dependentes

,chi2,pval,dof,dof_real
Delegacia_2anos,20.548447,3.890882e-04,4,2
Sexo,53.179802,1.079427e-09,6,3
Raça,10.612917,3.127663e-02,4,2
Cicatriz_BCG,19.234217,7.068855e-04,4,2
Febre,92.249234,4.381080e-19,4,2
Prisão_2anos,39.277414,6.105206e-08,4,2
Perda_peso_10percent,201.841399,1.509940e-42,4,2
Dispnéia,21.224927,1.671449e-03,6,3
Perda_de_apetite,13.526331,8.970844e-03,4,2
Semanas_tosse,96.249637,2.975851e-02,72,36


In [38]:
variaveis_independentes

,chi2,pval,dof,dof_real
CAGE,9.007891,0.060903,4,2
Expectoração,3.315559,0.506471,4,2
Fuma,9.230613,0.161018,6,3
Hemoptóicos,0.033715,0.999860,4,2
TTO_anterior_TB_triagem_enfermeiro,0.621002,0.960702,4,2
Internação_hospital_2anos,0.977081,0.913251,4,2
Hemoptise,7.689276,0.103647,4,2
Tosse,3.173160,0.529277,4,2
Dor_torácica,0.010302,0.999987,4,2
